# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the Used Cars dataset; there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* Please consider this notebook as an example and not to set specific requirements. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [1]:
import numpy as np
import pandas as pd
from pandarallel import pandarallel

In [2]:
%%capture
cd ..

In [3]:
import cleaner
import constants as const
import similarity as sim
import utils
import item_filters as itf

In [4]:
pandarallel.initialize()

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
df_train_raw = pd.read_csv(const.TRAIN_PATH)
df_train = cleaner.clean_preliminary(df_train_raw)

## Load the Data

For this example, we use a simplified version of the dataset with only 100 data sample, each with only 6 features

In [6]:
df_sample_raw = pd.read_csv(const.USED_CARS_SIMPLIFIED_PATH)
df_sample = cleaner.clean_preliminary(df_sample_raw)
df_sample.head()

,index,listing_id,make,power,engine_cap,mileage,price
0,0,1006025,bmw,135.0,1997.0,NaN,82300
1,1,994672,land rover,202.0,2993.0,25843.0,427900
2,2,921142,honda,95.6,1496.0,2000.0,109800
3,3,1008328,bmw,185.0,1998.0,57386.0,166600
4,4,1010661,NaN,96.0,1498.0,76000.0,59400


In [7]:
COLS_TO_SHOW = [
    'listing_id',
    'title',
    'make',
    'power',
    'engine_cap',
    'mileage',
    'price',
    'model',
    'description',
    'fuel_type',
    'accessories',
    'car_age',
]

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [8]:
DEFAULT_K = 10
DEFAULT_NOISE_LEVEL = 0.05
cache = {}


def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = kwargs.get('k', DEFAULT_K)
    noise_level = kwargs.get('noise_level', DEFAULT_NOISE_LEVEL)
    user_id = kwargs.get('user_id', None)
    user_pref = USER_PREFERENCES.get(user_id, {})
    assert k >= 1, 'k should be >= 1'
    assert noise_level >= 0.0, 'noise should not be negative'
    assert user_id is None or user_id in USER_PREFERENCES, \
        f'User ID not found, try one of {list(USER_PREFERENCES.keys())} or None'
    #####################################################
    # Get listing id similarity from cache if present, otherwise compute from scratch
    if row.listing_id in cache:
        sim_df = cache[row.listing_id]
    else:
        sim_df = sim.compute_similarities(pd.DataFrame([row]), df_train, is_test=True)
        cache[row.listing_id] = sim_df

    # Make noise similarity DataFrame
    noise = np.random.uniform(-noise_level, noise_level, size=len(df_train))
    sim_df_noisy = sim_df + noise
    
    # Get most similar
    most_similar_indices = np.argsort(-sim_df_noisy, axis=1).iloc[0]
    df_most_similar = df_train.iloc[most_similar_indices]
    
    # Filter based on user preferences (if any) and original listing id
    df_user = itf.filter_on_user_pref(user_pref, df_most_similar)
    df_result = df_user[df_user.listing_id != row.listing_id]

    # Return the dataset with the k recommendations
    return df_result.head(k)

## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Define User Preferences

Below, we define a user preferences dictionary where the key is the user id and the item is a dictionary of column-filter pairs specifying the item filter the user wants to apply for a given column.

It is assumed that if a user specifies some preferences, they *only* want to see cars that fullfill their preferences - i.e., we can do raw filtering based on their preferences.

The following item filters are available: 
* `RegexFilter(pattern: str)`
* `NumericalFilter(min_value: Optional[Number], max_value: Optional[Number])`
* `SetFilter(values: Set[Any])`
* `NotFilter(item_filter: ItemFilter)`

In [9]:
USER_PREFERENCES = {
    1: {
        'accessories': itf.RegexFilter(r'bluetooth'),
        'price': itf.NumericalFilter(max_value=110_000)
    },
    2: {
        'accessories': itf.RegexFilter(r'leather'),
        'car_age': itf.NumericalFilter(1, 3),
        'price': itf.NumericalFilter(min_value=50_000)
    }
}

### Pick a Sample Listing as Input

In [10]:
# Pick a row id of choice
#row_id = 10
#row_id = 20
row_id = 30
#row_id = 40
#row_id = 50

row = df_sample.iloc[row_id]
pd.DataFrame([row])

,index,listing_id,make,power,engine_cap,mileage,price
30,30,1019460,toyota,125.0,2362.0,28126.0,137300


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [11]:
# Number of similar rows to show
k = 5

# Noise level. The higher, the more noise the suggestions. If noise_level=0, then
# get_top_recommendations becomes deterministic
noise_level = 2.0

# Optional user id. Should be either None or an existing user_id - {1, 2} already defined
user_id = 1

df_recommendations = get_top_recommendations(row, k=k, noise_level=noise_level, user_id=user_id)
df_recommendations[COLS_TO_SHOW]

,listing_id,title,make,power,engine_cap,mileage,price,model,description,fuel_type,accessories,car_age
14344,1025806,Toyota Alphard 2.4A (New 10-yr COE),toyota,125.0,2362.0,96800.0,103200,alphard,1 owner unit serviced with borneo motors since...,petrol,"reverse camera, bluetooth, dual power sliding ...",11.0
7792,1008272,Toyota Vios 1.5A G,toyota,79.0,1496.0,39800.0,81300,vios,beautiful grey. agent maintained. car still un...,petrol,high g specs with gps/bluetooth/dvd function w...,2.0
12590,910038,Toyota Harrier 2.0A Elegance Panoramic,toyota,111.0,1986.0,83100.0,92200,harrier,first owner unit! interest rate 2.48% only! se...,petrol,"reverse camera, navi, bluetooth, dual zone aut...",5.0
9805,1021396,Toyota Prius Hybrid 1.8A S,toyota,90.0,1797.0,45000.0,80100,prius,1 owner only. low mileage. the prius is the mo...,petrol-electric,touch screen audio with bluetooth phone/audio ...,5.0
16116,1007931,Toyota Harrier 2.0A Premium Panoramic,toyota,111.0,1986.0,85888.0,75900,harrier,selling the cheapest and best condition harrie...,petrol,panoramic sunroof. premium upholstery electric...,6.0
